#**Maestría en Inteligencia Artificial Aplicada**
##**Curso: Inteligencia Artificial y Aprendizaje Automático**
###Tecnológico de Monterrey
###Prof Luis Eduardo Falcón Morales

## **Adtividad de la Semana 10**
###**Reducción de Dimensionalidad - Descomposición en Valores Singulares (SVD) y Sistemas de Recomendación**


**Nombres y matrículas de los integrantes del equipo:**

* Genaro Rodriguez Vazquez A01150931
* Juan Pablo Acosta López A01794035
* Maria Nelly Porras Alcantar A01793828
* Juan Carlos Torres Luna A01163204 


# **Objetivo**

---
El objetivo es diseñar tres sistemas de recomendación basados en la evaluación de los usarios a los 
restaurantes. Dicha evaluación será la general, la referente al servicio y la referente a la comida. 

---

En cada sección deberás incluir todas las líneas de código necesarias para responder a cada uno de los ejercicios.

In [1]:
# Incluye aquí todos módulos, librerías y paquetes que requieras.

import requests, zipfile
from io import BytesIO
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.metrics import fbeta_score, make_scorer
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_validate
from sklearn.model_selection import learning_curve, validation_curve
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import power_transform
from sklearn.impute import SimpleImputer
from sklearn.feature_selection._base import SelectorMixin
from sklearn.feature_extraction.text import _VectorizerMixin
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.dummy import DummyRegressor
from sklearn.linear_model import LinearRegression
from sklearn.neural_network import MLPClassifier
from sklearn.neural_network import MLPRegressor
from sklearn.decomposition import TruncatedSVD
import matplotlib.pyplot as plt 
import seaborn as sns
import math as math

In [2]:
#This method Imports zip directly
# importing necessary modules

#print('Downloading started')

#Defining the zip file URL
#url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/00232/RCdata.zip'

# Split URL to get the file name
#filename = url.split('/')[-1]

# Downloading the file by sending the request to the URL
#Para poder usar request necesitamos la libreria import requests, zipfile
from io import BytesIO

#req = requests.get(url)
#print('Downloading Completed')

# extracting the zip file contents
#zipfile= zipfile.ZipFile(BytesIO(req.content))
#zipfile.extractall('/content/sample_data/Restaurant_consumer')

Ratingdf = pd.read_csv('content/rating_final.csv',sep=',', encoding_errors='ignore')
Geodf = pd.read_csv('content/geoplaces2.csv',sep=',', encoding_errors='ignore')

In [3]:
Ratingdf

,userID,placeID,rating,food_rating,service_rating
0,U1077,135085,2,2,2
1,U1077,135038,2,2,1
2,U1077,132825,2,2,2
3,U1077,135060,1,2,2
4,U1068,135104,1,1,2
...,...,...,...,...,...
1156,U1043,132630,1,1,1
1157,U1011,132715,1,1,0
1158,U1068,132733,1,1,0
1159,U1068,132594,1,1,1


In [4]:
Geodf.head(2)

,placeID,latitude,longitude,the_geom_meter,name,address,city,state,country,fax,...,alcohol,smoking_area,dress_code,accessibility,price,url,Rambience,franchise,area,other_services
0,134999,18.915421,-99.184871,0101000020957F000088568DE356715AC138C0A525FC46...,Kiku Cuernavaca,Revolucion,Cuernavaca,Morelos,Mexico,?,...,No_Alcohol_Served,none,informal,no_accessibility,medium,kikucuernavaca.com.mx,familiar,f,closed,none
1,132825,22.147392,-100.983092,0101000020957F00001AD016568C4858C1243261274BA5...,puesto de tacos,esquina santos degollado y leon guzman,s.l.p.,s.l.p.,mexico,?,...,No_Alcohol_Served,none,informal,completely,low,?,familiar,f,open,none


In [5]:
dfrating = Ratingdf[['userID','placeID','rating']]
dfgeoplaces = Geodf[['placeID','name']]

relacion = pd.merge(dfrating,dfgeoplaces, on ='placeID')
relacion.head(2)

,userID,placeID,rating,name
0,U1077,135085,2,Tortas Locas Hipocampo
1,U1108,135085,1,Tortas Locas Hipocampo


#**Ejercicio-1.** 
---
Descarga los archivos y genera la matriz de utilidad en la cual los renglones son los usuarios 
(user_ID), las columnas los nombres de los restaurantes (name) y las entradas la evaluaciones 
generales dadas por los usuarios (rating). En particular indica el tamaño de dicha matriz y despliega 
sus primeros 10 renglones. 
---

In [6]:
rdf2 = pd.read_csv('content/rating_final.csv',sep=',', encoding_errors='ignore')
gdf2 = pd.read_csv('content/geoplaces2.csv',sep=',', encoding_errors='ignore')

dfrating = pd.merge(rdf2, gdf2, on='placeID') 
dfrating = dfrating.pivot_table(values='rating', index='userID', columns='name', fill_value=0)

print('El tamaño de la matriz de utilidad es: {}'.format(dfrating.shape))

dfrating.head(10)

El tamaño de la matriz de utilidad es: (138, 129)


name,Abondance Restaurante Bar,Arrachela Grill,Cabana Huasteca,Cafe Chaires,Cafeteria cenidet,Cafeteria y Restaurant El Pacifico,Carls Jr,Carnitas Mata Calle 16 de Septiembre,Carreton de Flautas y Migadas,Cenaduria El Rincón de Tlaquepaque,...,puesto de tacos,rockabilly,shi ro ie,sirloin stockade,tacos abi,tacos de barbacoa enfrente del Tec,tacos de la estacion,tacos los volcanes,tortas hawai,vips
userID,,,,,,,,,,,,,,,,,,,,,
U1001,0,0,0,0,0,0,0,0,0,0,...,2,0,0,0,0,0,0,0,0,0
U1002,0,0,0,0,0,0,0,0,0,0,...,2,0,0,0,0,0,0,0,0,0
U1003,0,0,2,0,0,0,0,0,0,0,...,2,2,0,0,0,0,0,0,0,0
U1004,0,0,0,0,0,2,0,0,0,0,...,0,0,0,0,0,0,0,2,0,0
U1005,0,0,0,0,0,2,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
U1006,0,0,0,1,0,0,0,0,0,0,...,1,0,2,0,0,0,0,0,0,0
U1007,0,0,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
U1008,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
U1009,0,0,1,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0


#**Ejercicio-2.**
---
Con el objetivo de obtener la información con respecto a los vectores latentes de los restaurantes, 
obtener dicha factorización SVD utilizando todas las componentes posibles, es decir, usando todos 
sus valores singulares. En particular indica la cantidad de valores singulares obtenidos. 
---

In [7]:
X_rating = dfrating.T
X_rating.shape

#Utilizando todos los componentes
num_svw = 137

SVD_rating = TruncatedSVD(n_components=num_svw, random_state=1)

resultant_matrix_rating = SVD_rating.fit_transform(X_rating)
resultant_matrix_rating.shape

(129, 129)

#**Ejercicio-3.**
---
Usando la métrica de correlación de Pearon, obtener las 10 mejores recomendaciones de este 
modelo no supervisado con base a la información de alguien que evaluó bien de manera general al 
“Subway”. Despliega también el valor numérico de la correlación de Pearson de cada una de estas 10 
recomendaciones. NOTA: No debe estar incluido “Subway” dentro de las recomendaciones.
---

In [8]:
corrMxw = np.corrcoef(resultant_matrix_rating, )
me_gusta = "Subway"

names = dfrating.columns
names_list = list(names)
id_megusta = names_list.index(me_gusta)

corr_recomienda = corrMxw[id_megusta]


RecomendationsMatrix = [[],[]]

for x in range(0,len(names)):
    if corr_recomienda[x] > .20:
        RecomendationsMatrix[0].append([names_list[x]])
        RecomendationsMatrix[1].append((corr_recomienda[x]))

Recomendationsdf = pd.DataFrame(RecomendationsMatrix).T
Recomendationsdf.columns = ['Restaurante recomendado', 'Corr de Pearson']
Recomendationsdf

Top10df = Recomendationsdf.sort_values(by='Corr de Pearson', ascending=False)
#Top10df['Corr de Pearson'] = Top10df['Corr de Pearson'].astype(float)
Top10df = Top10df.loc[(Top10df['Corr de Pearson'] < 1)].head(10)
Top10df.reset_index(inplace=True)
print("Top10 utilizando todos los componentes:")
Top10df
  

Top10 utilizando todos los componentes:


,index,Restaurante recomendado,Corr de Pearson
0,5,[Restaurant Teely],0.609521
1,11,[Vips],0.506278
2,8,[Rincon del Bife],0.429824
3,6,[Restaurant and Bar and Clothesline Carlos N C...,0.425467
4,9,[Sanborns Casa Piedra],0.417219
5,2,[Mariscos Tia Licha],0.388178
6,0,[Chilis Cuernavaca],0.30109
7,3,[Mikasa],0.301002
8,7,[Restaurant los Pinos],0.285149
9,1,[Log Yin],0.234634


#**Ejercicio-4.**
---
Disminuye ahora la dimensión de la factorización con un número de componentes tal, que las 
recomendaciones con esta menor cantidad de información coincidan en al menos 9 de las 10 
recomendaciones dadas con todos los valores singulares. Indica la cantidad de valores singulares 
encontrado y la cantidad de información que contienen.
---

In [9]:
SVD_Rating = TruncatedSVD(n_components=137, random_state=1)  
SVD_Rating.fit(X_rating)

num_sv = 5

print('Cantidad de información simplificada con los primeros %d vectores singulares:' % num_sv)
print('%.1f%%' %  (100 * (1- (SVD_Rating.singular_values_[0:num_sv]).sum() / (SVD_Rating.singular_values_).sum())))


SVD_Rating = TruncatedSVD(n_components=num_sv, random_state=1)

resultant_matrix = SVD_Rating.fit_transform(X_rating)
#resultant_matrix.shape
corrMx = np.corrcoef(resultant_matrix, )
#corrMx.shape
me_gusta = "Subway"

names = dfrating.columns
names_list = list(names)
id_megusta = names_list.index(me_gusta)

corr_recomienda = corrMx[id_megusta]

RecomendationsMatrix = [[],[]]

for x in range(0,len(names)):
    if corr_recomienda[x] > .17:
        RecomendationsMatrix[0].append([names_list[x]])
        RecomendationsMatrix[1].append((corr_recomienda[x]))

Recomendationsdf = pd.DataFrame(RecomendationsMatrix).T
Recomendationsdf.columns = ['Restaurante recomendado', 'Corr de Pearson']
Recomendationsdf

Top10df = Recomendationsdf.sort_values(by='Corr de Pearson', ascending=False)
#Top10df['Corr de Pearson'] = Top10df['Corr de Pearson'].astype(float)
Top10df = Top10df.loc[(Top10df['Corr de Pearson'] < 1)].head(10)
Top10df.reset_index(inplace=True)
print("Top10 utilizando solo %d componentes:" % num_sv)
Top10df

Cantidad de información simplificada con los primeros 5 vectores singulares:
85.5%
Top10 utilizando solo 5 componentes:


,index,Restaurante recomendado,Corr de Pearson
0,4,[Chilis Cuernavaca],1.0
1,22,[Restaurant Teely],1.0
2,25,[Rincon del Bife],1.0
3,17,[Paniroles],1.0
4,23,[Restaurant and Bar and Clothesline Carlos N C...,0.999998
5,14,[Mariscos Tia Licha],0.999998
6,16,[Mikasa],0.999997
7,31,[Vips],0.999996
8,12,[Log Yin],0.999996
9,26,[Sanborns Casa Piedra],0.99999


    En base a prueba y error nos percatamos que un mínimo de 5 valores singulares nos arrojan 9 de las 10 recomendaciones que obtenemos al utilizar todos los valores singulares.
    En pocas palabras, logramos reducir de 137 a solo 5 valores singulares, a costa de tener 1 de los 10 recomendaciones del top10 erronea, esto se traduce a reducir en un 85.5% la cantidad de variables de entrada necesarias para la prediccion.

#**Ejercicio-5.**
---
Repite el ejercicio con el restaurante “"tacos de barbacoa enfrente del Tec". Incluye tus conclusiones.
---

In [10]:
corrMxw = np.corrcoef(resultant_matrix_rating, )
me_gusta = "tacos de barbacoa enfrente del Tec"

names = dfrating.columns
names_list = list(names)
id_megusta = names_list.index(me_gusta)

corr_recomienda = corrMxw[id_megusta]


RecomendationsMatrix = [[],[]]

for x in range(0,len(names)):
    if corr_recomienda[x] > .15:
        RecomendationsMatrix[0].append([names_list[x]])
        RecomendationsMatrix[1].append((corr_recomienda[x]))

Recomendationsdf = pd.DataFrame(RecomendationsMatrix).T
Recomendationsdf.columns = ['Restaurante recomendado', 'Corr de Pearson']
Recomendationsdf

Top10df = Recomendationsdf.sort_values(by='Corr de Pearson', ascending=False)
#Top10df['Corr de Pearson'] = Top10df['Corr de Pearson'].astype(float)
Top10df = Top10df.loc[(Top10df['Corr de Pearson'] < 1)].head(10)
Top10df.reset_index(inplace=True)
print("Top10 utilizando todos los componentes:")
Top10df
  

Top10 utilizando todos los componentes:


,index,Restaurante recomendado,Corr de Pearson
0,11,[vips],0.817189
1,9,[tacos abi],0.665305
2,6,[little pizza Emilio Portes Gil],0.653849
3,3,[Taqueria EL amigo ],0.437184
4,8,[puesto de gorditas],0.412765
5,0,[Carreton de Flautas y Migadas],0.407795
6,2,[Little Cesarz],0.258479
7,7,[palomo tec],0.195174
8,1,[Hamburguesas La perica],0.19112
9,4,[carnitas mata calle Emilio Portes Gil],0.170939


In [11]:
SVD_Rating = TruncatedSVD(n_components=137, random_state=1)  
SVD_Rating.fit(X_rating)

num_sv = 78

print('Cantidad de información simplificada con los primeros %d vectores singulares:' % num_sv)
print('%.1f%%' %  (100 * (1- (SVD_Rating.singular_values_[0:num_sv]).sum() / (SVD_Rating.singular_values_).sum())))


SVD_Rating = TruncatedSVD(n_components=num_sv, random_state=1)

resultant_matrix = SVD_Rating.fit_transform(X_rating)
#resultant_matrix.shape
corrMx = np.corrcoef(resultant_matrix, )
#corrMx.shape
me_gusta = "tacos de barbacoa enfrente del Tec"

names = dfrating.columns
names_list = list(names)
id_megusta = names_list.index(me_gusta)

corr_recomienda = corrMx[id_megusta]

RecomendationsMatrix = [[],[]]

for x in range(0,len(names)):
    if corr_recomienda[x] > .17:
        RecomendationsMatrix[0].append([names_list[x]])
        RecomendationsMatrix[1].append((corr_recomienda[x]))

Recomendationsdf = pd.DataFrame(RecomendationsMatrix).T
Recomendationsdf.columns = ['Restaurante recomendado', 'Corr de Pearson']
Recomendationsdf

Top10df = Recomendationsdf.sort_values(by='Corr de Pearson', ascending=False)
#Top10df['Corr de Pearson'] = Top10df['Corr de Pearson'].astype(float)
Top10df = Top10df.loc[(Top10df['Corr de Pearson'] < 0.999999999)].head(10)
Top10df.reset_index(inplace=True)
print("Top10 utilizando solo %d componentes:" % num_sv)
Top10df

Cantidad de información simplificada con los primeros 78 vectores singulares:
10.3%
Top10 utilizando solo 78 componentes:


,index,Restaurante recomendado,Corr de Pearson
0,7,[little pizza Emilio Portes Gil],0.866305
1,11,[vips],0.858399
2,9,[tacos abi],0.846151
3,8,[puesto de gorditas],0.57049
4,4,[Taqueria EL amigo ],0.521119
5,0,[Carreton de Flautas y Migadas],0.506756
6,3,[Little Cesarz],0.316038
7,6,[carnitas_mata],0.266037
8,2,[Hamburguesas La perica],0.203174
9,5,[carnitas mata calle Emilio Portes Gil],0.178717


    En este estudio a prueba y error nos percatamos que esta vez solo logramos reducir un 10.3% la cantidad de variables de entrada necesarias para una prediccion que arroje al menos 9 de las 10 recomendaciones que obtenemos al utilizar todos los valores singulares.

    En pocas palabras, logramos reducir de 137 a solo 78 valores singulares, a costa de tener 1 de los 10 recomendaciones del top10 erronea esta vez.

#**Ejercicio-6.**
---
Repite los ejercicios 1 a 4 generando ahora la matriz de utilidad en la cual los renglones son los 
usuarios (user_ID), las columnas los nombres de los restaurantes (name) y las entradas la 
evaluación dada a la comida (food_rating). Incluye tus conclusiones.
---

In [12]:
rdf2 = pd.read_csv('content/rating_final.csv',sep=',', encoding_errors='ignore')
gdf2 = pd.read_csv('content/geoplaces2.csv',sep=',', encoding_errors='ignore')

dffood = pd.merge(rdf2, gdf2, on='placeID') 
dffood = dffood.pivot_table(values='food_rating', index='userID', columns='name', fill_value=0)

print('El tamaño de la matriz es: {}'.format(dffood.shape))

dffood.head(10)

El tamaño de la matriz es: (138, 129)


name,Abondance Restaurante Bar,Arrachela Grill,Cabana Huasteca,Cafe Chaires,Cafeteria cenidet,Cafeteria y Restaurant El Pacifico,Carls Jr,Carnitas Mata Calle 16 de Septiembre,Carreton de Flautas y Migadas,Cenaduria El Rincón de Tlaquepaque,...,puesto de tacos,rockabilly,shi ro ie,sirloin stockade,tacos abi,tacos de barbacoa enfrente del Tec,tacos de la estacion,tacos los volcanes,tortas hawai,vips
userID,,,,,,,,,,,,,,,,,,,,,
U1001,0,0,0,0,0,0,0,0,0,0,...,2,0,0,0,0,0,0,0,0,0
U1002,0,0,0,0,0,0,0,0,0,0,...,2,0,0,0,0,0,0,0,0,0
U1003,0,0,2,0,0,0,0,0,0,0,...,2,2,0,0,0,0,0,0,0,0
U1004,0,0,0,0,0,2,0,0,0,0,...,0,0,0,0,0,0,0,2,0,0
U1005,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
U1006,0,0,0,0,0,0,0,0,0,0,...,1,0,2,0,0,0,0,0,0,0
U1007,0,0,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
U1008,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
U1009,0,0,2,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0


In [13]:
X_food = dffood.T
X_food.shape

#Utilizando todos los componentes
num_svw = 137

SVD_rating = TruncatedSVD(n_components=num_svw, random_state=1)

resultant_matrix_food = SVD_rating.fit_transform(X_food)
resultant_matrix_food.shape

(129, 129)

In [14]:
corrMxw = np.corrcoef(resultant_matrix_food, )
me_gusta = "Subway"

names = dffood.columns
names_list = list(names)
id_megusta = names_list.index(me_gusta)

corr_recomienda = corrMxw[id_megusta]


RecomendationsMatrix = [[],[]]

for x in range(0,len(names)):
    if corr_recomienda[x] > .23:
        RecomendationsMatrix[0].append([names_list[x]])
        RecomendationsMatrix[1].append((corr_recomienda[x]))

Recomendationsdf = pd.DataFrame(RecomendationsMatrix).T
Recomendationsdf.columns = ['Restaurante recomendado', 'Corr de Pearson']
Recomendationsdf

Top10df = Recomendationsdf.sort_values(by='Corr de Pearson', ascending=False)
#Top10df['Corr de Pearson'] = Top10df['Corr de Pearson'].astype(float)
Top10df = Top10df.loc[(Top10df['Corr de Pearson'] < 1)].head(10)
Top10df.reset_index(inplace=True)
print("Top10 utilizando todos los componentes:")
Top10df
  

Top10 utilizando todos los componentes:


,index,Restaurante recomendado,Corr de Pearson
0,6,[Restaurant Teely],0.816413
1,7,[Restaurant and Bar and Clothesline Carlos N C...,0.46895
2,11,[Vips],0.442148
3,2,[Mariscos Tia Licha],0.415222
4,8,[Rincon del Bife],0.381819
5,9,[Sanborns Casa Piedra],0.365657
6,5,[Restaurant Las Mananitas],0.331707
7,3,[Paniroles],0.312082
8,1,[Kiku Cuernavaca],0.250357
9,4,[Restaurant Familiar El Chino],0.232708


In [15]:
SVD_Rating = TruncatedSVD(n_components=137, random_state=1)  
SVD_Rating.fit(X_food)

num_sv = 72

print('Cantidad de información simplificada con los primeros %d vectores singulares:' % num_sv)
print('%.1f%%' %  (100 * (1- (SVD_Rating.singular_values_[0:num_sv]).sum() / (SVD_Rating.singular_values_).sum())))


SVD_Rating = TruncatedSVD(n_components=num_sv, random_state=1)

resultant_matrix = SVD_Rating.fit_transform(X_food)
#resultant_matrix.shape
corrMx = np.corrcoef(resultant_matrix, )
#corrMx.shape
me_gusta = "Subway"

names = dffood.columns
names_list = list(names)
id_megusta = names_list.index(me_gusta)

corr_recomienda = corrMx[id_megusta]

RecomendationsMatrix = [[],[]]

for x in range(0,len(names)):
    if corr_recomienda[x] > .23:
        RecomendationsMatrix[0].append([names_list[x]])
        RecomendationsMatrix[1].append((corr_recomienda[x]))

Recomendationsdf = pd.DataFrame(RecomendationsMatrix).T
Recomendationsdf.columns = ['Restaurante recomendado', 'Corr de Pearson']
Recomendationsdf

Top10df = Recomendationsdf.sort_values(by='Corr de Pearson', ascending=False)
#Top10df['Corr de Pearson'] = Top10df['Corr de Pearson'].astype(float)
Top10df = Top10df.loc[(Top10df['Corr de Pearson'] < 0.999999999)].head(10)
Top10df.reset_index(inplace=True)
print("Top10 utilizando solo %d componentes:" % num_sv)
Top10df

Cantidad de información simplificada con los primeros 72 vectores singulares:
13.5%
Top10 utilizando solo 72 componentes:


,index,Restaurante recomendado,Corr de Pearson
0,6,[Restaurant Teely],0.881193
1,7,[Restaurant and Bar and Clothesline Carlos N C...,0.513714
2,8,[Rincon del Bife],0.474916
3,11,[Vips],0.448744
4,1,[Mariscos Tia Licha],0.435959
5,9,[Sanborns Casa Piedra],0.384273
6,5,[Restaurant Las Mananitas],0.334512
7,3,[Paniroles],0.31729
8,2,[Mikasa],0.268565
9,4,[Restaurant Familiar El Chino],0.25213


    En base a prueba y error nos percatamos que un mínimo de 72 valores singulares nos arrojan 9 de las 10 recomendaciones que obtenemos al utilizar todos los valores singulares.
    En pocas palabras, logramos reducir de 137 a solo 5 valores singulares, a costa de tener 1 de los 10 recomendaciones del top10 erronea, esto se traduce a reducir en un 13.5% la cantidad de variables de entrada necesarias para la prediccion.
    

#**Ejercicio-7.**

---
Repite los ejercicios 1 a 4 generando ahora la matriz de utilidad en la cual los renglones son los usuarios (user_ID), las columnas los nombres de los restaurantes (name) y las entradas la evaluación dada al servicio (service_rating). Incluye tus conclusiones. 

---

In [16]:
rdf2 = pd.read_csv('content/rating_final.csv',sep=',', encoding_errors='ignore')
gdf2 = pd.read_csv('content/geoplaces2.csv',sep=',', encoding_errors='ignore')

dfservice = pd.merge(rdf2, gdf2, on='placeID') 
dfservice = dfservice.pivot_table(values='service_rating', index='userID', columns='name', fill_value=0)

print('El tamaño de la matriz es: {}'.format(dfservice.shape))

dfservice.head(10)

El tamaño de la matriz es: (138, 129)


name,Abondance Restaurante Bar,Arrachela Grill,Cabana Huasteca,Cafe Chaires,Cafeteria cenidet,Cafeteria y Restaurant El Pacifico,Carls Jr,Carnitas Mata Calle 16 de Septiembre,Carreton de Flautas y Migadas,Cenaduria El Rincón de Tlaquepaque,...,puesto de tacos,rockabilly,shi ro ie,sirloin stockade,tacos abi,tacos de barbacoa enfrente del Tec,tacos de la estacion,tacos los volcanes,tortas hawai,vips
userID,,,,,,,,,,,,,,,,,,,,,
U1001,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
U1002,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
U1003,0,0,2,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,0,0
U1004,0,0,0,0,0,2,0,0,0,0,...,0,0,0,0,0,0,0,2,0,0
U1005,0,0,0,0,0,2,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
U1006,0,0,0,1,0,0,0,0,0,0,...,1,0,2,0,0,0,0,0,0,0
U1007,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
U1008,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
U1009,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [17]:
X_service = dfservice.T
X_service.shape

#Utilizando todos los componentes
num_svw = 137

SVD_rating = TruncatedSVD(n_components=num_svw, random_state=1)

resultant_matrix_service = SVD_rating.fit_transform(X_service)
resultant_matrix_service.shape

(129, 129)

In [18]:
corrMxw = np.corrcoef(resultant_matrix_service, )
me_gusta = "Subway"

names = dfservice.columns
names_list = list(names)
id_megusta = names_list.index(me_gusta)

corr_recomienda = corrMxw[id_megusta]


RecomendationsMatrix = [[],[]]

for x in range(0,len(names)):
    if corr_recomienda[x] > .17:
        RecomendationsMatrix[0].append([names_list[x]])
        RecomendationsMatrix[1].append((corr_recomienda[x]))

Recomendationsdf = pd.DataFrame(RecomendationsMatrix).T
Recomendationsdf.columns = ['Restaurante recomendado', 'Corr de Pearson']
Recomendationsdf

Top10df = Recomendationsdf.sort_values(by='Corr de Pearson', ascending=False)
#Top10df['Corr de Pearson'] = Top10df['Corr de Pearson'].astype(float)
Top10df = Top10df.loc[(Top10df['Corr de Pearson'] < 1)].head(10)
Top10df.reset_index(inplace=True)
print("Top10 utilizando todos los componentes:")
Top10df
  

Top10 utilizando todos los componentes:


,index,Restaurante recomendado,Corr de Pearson
0,6,[Restaurant Teely],0.772973
1,7,[Restaurant and Bar and Clothesline Carlos N C...,0.600161
2,8,[Rincon del Bife],0.518731
3,10,[Vips],0.4212
4,3,[Paniroles],0.365934
5,0,[Chilis Cuernavaca],0.230698
6,5,[Restaurant Las Mananitas],0.226266
7,1,[Giovannis],0.19276
8,4,[Restaurant Familiar El Chino],0.182869
9,2,[Kiku Cuernavaca],0.17874


In [25]:
SVD_Rating = TruncatedSVD(n_components=137, random_state=1)  
SVD_Rating.fit(X_service)

num_sv = 48

print('Cantidad de información simplificada con los primeros %d vectores singulares:' % num_sv)
print('%.1f%%' %  (100 * (1- (SVD_Rating.singular_values_[0:num_sv]).sum() / (SVD_Rating.singular_values_).sum())))


SVD_Rating = TruncatedSVD(n_components=num_sv, random_state=1)

resultant_matrix = SVD_Rating.fit_transform(X_service)
#resultant_matrix.shape
corrMx = np.corrcoef(resultant_matrix, )
#corrMx.shape
me_gusta = "Subway"

names = dfservice.columns
names_list = list(names)
id_megusta = names_list.index(me_gusta)

corr_recomienda = corrMx[id_megusta]

RecomendationsMatrix = [[],[]]

for x in range(0,len(names)):
    if corr_recomienda[x] > .17:
        RecomendationsMatrix[0].append([names_list[x]])
        RecomendationsMatrix[1].append((corr_recomienda[x]))

Recomendationsdf = pd.DataFrame(RecomendationsMatrix).T
Recomendationsdf.columns = ['Restaurante recomendado', 'Corr de Pearson']
Recomendationsdf

Top10df = Recomendationsdf.sort_values(by='Corr de Pearson', ascending=False)
#Top10df['Corr de Pearson'] = Top10df['Corr de Pearson'].astype(float)
Top10df = Top10df.loc[(Top10df['Corr de Pearson'] < 1)].head(10)
Top10df.reset_index(inplace=True)
print("Top10 utilizando solo %d componentes:" % num_sv)
Top10df

Cantidad de información simplificada con los primeros 48 vectores singulares:
29.1%
Top10 utilizando solo 48 componentes:


,index,Restaurante recomendado,Corr de Pearson
0,7,[Restaurant Teely],0.95779
1,8,[Restaurant and Bar and Clothesline Carlos N C...,0.875834
2,9,[Rincon del Bife],0.698045
3,4,[Paniroles],0.546462
4,11,[Vips],0.462974
5,1,[Giovannis],0.288172
6,6,[Restaurant Las Mananitas],0.23372
7,0,[Chilis Cuernavaca],0.233651
8,2,[Kiku Cuernavaca],0.184832
9,3,[Mariscos Tia Licha],0.184168


    En base a prueba y error nos percatamos que un mínimo de 48 valores singulares nos arrojan 9+ de las 10 recomendaciones que obtenemos al utilizar todos los valores singulares.
    En pocas palabras, logramos reducir de 137 a solo 5 valores singulares, a costa de tener 1 de los 10 recomendaciones del top10 erronea, esto se traduce a reducir en un 29.1% la cantidad de variables de entrada necesarias para la prediccion.
    

#**Ejercicio-8.**

**Conclusiones finales:**

No es necesarios usar todos los valores singulares, se puede optimizar el modelo para obtener la misma información con menos valores.

En el caso estudiado "Subway" se observa que hay un poco de discrepancia entre las 3 listas de cada uno de los diferentes ratings, sin embargo, entre ellas hay una pequeño subconjunto que coincide en 2 o 3 como lo son "VIPS, Teeley, Chillis" entre otros, por lo tanto si este fuera un sistema de recomendación en producción, se debería de tomar en cuenta las 3 listas y dar únicamente los restaurantes que coinciden en 2 o 3 listas del mismo restaurante o en su defecto se presentarían las 2 listas de coincidencia de food y service y se dejaría a criterio del usuario a utilizar

###**Fin de la Actividad de la semana 10 - Sistemas de Recomendación y Reducción de Dimensionalidad con SVD.**